# Save mat files to memmaps

In [1]:
# A bit of setup

import numpy as np
from scipy.io import loadmat
import os.path as path
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Load Data

## Rates

In [3]:
rates_dir = '/Users/lmcintosh/Git/deepRGC/datasets/white_noise/'
filename  = 'rates.mat'

rates_mat = loadmat(rates_dir + filename, squeeze_me=True, struct_as_record=False)
rates = rates_mat['rates']

In [4]:
rates.shape

(299890, 11)

## Stimulus

In [5]:
stim_dir = '/Users/lmcintosh/Git/deepRGC/datasets/white_noise/'
filename1 = 'stim0a.mat'
filename2 = 'stim0b.mat'

stim1_mat = loadmat(stim_dir + filename1, squeeze_me=True, struct_as_record=False)
stim2_mat = loadmat(stim_dir + filename2, squeeze_me=True, struct_as_record=False)

stim = np.concatenate((stim1_mat['stimulus'], stim2_mat['stimulus']), axis=2)

## Create memmap

In [25]:
filename = path.join(rates_dir, 'rates.dat')
fp = np.memmap(filename, dtype=str(rates.dtype), mode='w+', shape=rates.shape)
fp[:] = rates[:]

In [26]:
del fp

In [28]:
np.mean(newfp, axis=0)/.01

memmap([ 2.03874754,  1.99106339,  3.42625629,  4.60768949,  0.77595118,
        3.24452299,  4.75407649,  4.42829037,  4.59001634,  5.71242789,
        3.87708827])

#### Create uint8 version of stim

In [ ]:
filename = path.join(stim_dir, 'stim.dat')
fp = np.memmap(filename, dtype=str(stim.dtype), mode='w+', shape=stim.shape)
fp[:] = stim[:]

#### Create normalized (and float32) version of stim

In [4]:
stim.shape

(100, 100, 299890)

In [3]:
mean_pixel = np.mean(stim, dtype='float32')

In [5]:
filename = path.join(stim_dir, 'stim_norm.dat')
fp = np.memmap(filename, dtype='float32', mode='w+', shape=stim.shape)
#fp[:] = stim[:].astype('float32') - mean_pixel

In [12]:
batchSize = 10000
for it in xrange(stim.shape[2] / batchSize):
    fp[:,:,it*batchSize:(it+1)*batchSize] = stim[:,:,it*batchSize:(it+1)*batchSize].astype('float32')
fp[:,:,(it+1)*batchSize:] = stim[:,:,(it+1)*batchSize:].astype('float32')

In [13]:
np.min(fp)

memmap(0.0, dtype=float32)

In [14]:
np.max(fp)

memmap(1.0, dtype=float32)

In [15]:
np.mean(fp, dtype='float32')

0.0055944566

In [33]:
np.mean(fp[:,:,100000]) # something is seriously messed up with np.mean on 3d memmap arrays

0.50099999

In [34]:
fp -= 0.5

In [49]:
np.mean(fp[:,:,295000])

0.0041

In [52]:
np.min(fp[:,:,29500])

memmap(-0.5, dtype=float32)

In [53]:
del fp

In [6]:
metadata = {}
metadata['rates_shape'] = rates.shape
metadata['stim_shape'] = stim.shape
metadata['stim_norm_shape'] = stim.shape
metadata['rates_dtype'] = str(rates.dtype)
metadata['stim_dtype'] = str(stim.dtype)
metadata['stim_norm_dtype'] = 'float32'
filename = path.join(stim_dir, 'metadata.npz')
np.savez(filename, metadata=metadata)

## Load memmap

In [34]:
filename = path.join(rates_dir, 'rates.dat')
rates_mmap = np.memmap(filename, dtype=str(rates.dtype), mode='r', shape=rates.shape)

In [35]:
np.mean(rates_mmap, axis=0)/.01

memmap([ 2.03874754,  1.99106339,  3.42625629,  4.60768949,  0.77595118,
        3.24452299,  4.75407649,  4.42829037,  4.59001634,  5.71242789,
        3.87708827])

In [ ]:
filename = path.join(stim_dir, 'stim_norm.dat')
stim_mmap = np.memmap(filename, dtype='float32', mode='r', shape=stim.shape)